# Aujourd'hui on roule sur les mecs de l'ENS


https://challengedata.ens.fr/en/challenge/39/prediction_of_transaction_claims_status.html

# Imports des librairies de bases

On ajoutera celles qui manquent au fur et à mesure de nos besoins

In [3]:
# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals

# Common imports
import numpy as np
import pandas as pd
import os, gc

# Définition de la seed pour le random

Très important pour qu'on voit les mêmes choses entre nos deux ordis

In [4]:
RANDOM_SEED = 42;
np.random.seed(RANDOM_SEED)

# Définition des paramètres pour Matplot

Rien de bien intéréssant

In [5]:
# To plot pretty figures
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# Set des variables globales

Attention, je n'utilise les variables globales pour la gestion des fichiers. Sinon, c'est mort

In [6]:
# Where to save the figures
PROJECT_ROOT_DIR = "."
DATA_PATH = os.path.join(PROJECT_ROOT_DIR, "data")

# Fonction pour load les libraires

En vrai, on a juste besoin de pd.read_csv, mais c'était pour faire joli

In [7]:
def load_data(file,data_path=DATA_PATH, sep=','):
    csv_path = os.path.join(data_path, file)
    return pd.read_csv(csv_path, sep)

# On load les jeux de données

Seulement le train (input_train) pour le moment. Et le "Y" des lignes dans le fichier avec le long nom.
Je ne sais pas pourquoi ils ont tout divisé, mais peût être pour filter les mecs qui ne savent pas fire des jointures.

In [8]:
TX_data = load_data(file = "input_train.csv");
STA_data = load_data(file = "challenge_output_data_training_file_prediction_of_transaction_claims_status.csv", sep=';');

# Jointure entre les X et Y

In [9]:
TX_data = pd.merge(TX_data, STA_data, left_index=True, right_index=True)
TX_data.drop(["ID_y","ID_x"],inplace=True,axis=1)

del STA_data #Je supprime toujours les variables inutiles pour liberer ma RAM (qui est très faible)

### Je laisse ne commentaire les fonctions qui m'ont été utiles pour analyser la BDD

In [10]:
#TX_data.head() 

In [11]:
#TX_data["CLAIM_TYPE"].value_counts().head(10).plot.bar()

In [12]:
#TX_data.info()

# Fonction de mise en forme (attention, sportive)

Dans la BDD, j'ai vu pas mal de truc du genre "20<50", inutilisables. Donc, j'ai voulu splité pour faire une colonne avec le "20" (MIN), et une autre avec le "50" (MAX).
Donc j'ai fait cette fonction qui prend en entrée la BDD, une liste de noms de colones à traiter, et modifie la base de donnée directement.

In [53]:
class Splitter:
    
    def __init__(self,list_col):
        self.list_work=list_col;
    
    def fit(self, *_):
        return self
    
    def transform(self, data):
        X=data.copy()
        #X=X[self.list_work]
        for colname in self.list_work: # Pour chaque nom de colonne spécifié
            toto= data[colname].str.split('<', 1, expand=True) # Split sur le char "<" de la colonne. Cette fonction crée 2 colonnes : Une avec ce qu'il y a a gauche du split, une autre à droite
            X[colname+"_MIN"] = toto[0] # On prend la partie gauche pour faire le MIN
            X[colname+"_MAX"] = toto[1] # La droite pour le MAX
            X[colname+"_MIN"]=X[colname+"_MIN"].str.replace('>',"") # Parfois il y a le signe ">", donc je le vire simplement
            X.drop(colname, axis=1, inplace=True) # Et je drop la colonne inutile maintenant. (Inplace = True veut dire modification de la BDD originale)
        #X.replace("", np.nan, inplace=True)
        #X.fillna(value=0, inplace=True)
        return X;

In [54]:
list_col_split=["WARRANTIES_PRICE",'SELLER_SCORE_COUNT','ITEM_PRICE','PURCHASE_COUNT','SHIPPING_PRICE']

In [55]:
split=Splitter(list_col=list_col_split)
del list_col_split

In [56]:
class DateSplit:
    
    def __init__(self):
        pass
    
    def fit(self, *_):
        return self
    
    def transform(self, data):
        X=data.copy()
        #X=X['BUYING_DATE']
        toto= X['BUYING_DATE'].str.split('/', 1, expand=True)
        X['BUYING_DATE_Month'] = toto[0]
        X['BUYING_DATE_Year'] = toto[1]
        X.drop('BUYING_DATE', axis=1, inplace=True)
        return X;

# Feature Engineering

Là, faut être créatif, et aider l'algorithme à voir des choses, et se focaliser sur d'autres.

Je vais rarement drop des colonnes, car à ce moment là, je ne sais pas ce qui est utile ou pas

## A une garantie

In [25]:
class WarrantFlag:
    
    def __init__(self):
        pass
    
    def fit(self, *_):
        return self
    
    def transform(self, data):
        X=data.copy();
        X=X['WARRANTIES_FLG'];
        X["WARRANTIES_FLG"] = X["WARRANTIES_FLG"].map({True : 1, False : 0 })
        return X;

## Taux de couverture de la garantie

In [ ]:
# Beaucoup de valeurs en NaN car cela correspond à "pas de garantie". 
# Donc je dis que si il n'y a pas de garantie, elle est à zéro

TX_data.WARRANTIES_PRICE_MAX.fillna(value=0, inplace=True)
TX_data.WARRANTIES_PRICE_MIN.fillna(value=0, inplace=True)

In [ ]:
class WarrantCoverage:
    
    def __init__(self):
        pass;
    
    def fit(self, *_):
        return self
    
    def transform(self, data):
        X=data.copy();
        keep_list=["WARRANTIES_PRICE_MIN","WARRANTIES_PRICE_MAX","ITEM_PRICE_MIN","ITEM_PRICE_MAX"]
        X=X[keep_list]
        
        X["WARRANTIES_COVERAGE_MIN"]=X["WARRANTIES_PRICE_MIN"]/(X["ITEM_PRICE_MIN"]+0.1)
        X["WARRANTIES_COVERAGE_MAX"]=X["WARRANTIES_PRICE_MAX"]/(X["ITEM_PRICE_MAX"]+0.1)
        X.drop(keep_list, axis=1, inplace=True)
        return X;

In [23]:
TX_data.head(5)

,SHIPPING_MODE,SHIPPING_PRICE,WARRANTIES_FLG,WARRANTIES_PRICE,CARD_PAYMENT,COUPON_PAYMENT,RSP_PAYMENT,WALLET_PAYMENT,PRICECLUB_STATUS,REGISTRATION_DATE,...,BUYER_DEPARTMENT,BUYING_DATE,SELLER_SCORE_COUNT,SELLER_SCORE_AVERAGE,SELLER_COUNTRY,SELLER_DEPARTMENT,PRODUCT_TYPE,PRODUCT_FAMILY,ITEM_PRICE,CLAIM_TYPE
0,NORMAL,NaN,False,NaN,1,0,1,0,UNSUBSCRIBED,2015,...,34,3/2017,10000<100000,46.0,"FRANCE, METROPOLITAN",61,CELLPHONE_ACCESSORY,ELECTRONICS,<10,DAMAGED
1,NORMAL,NaN,False,NaN,1,0,0,0,UNSUBSCRIBED,2013,...,77,8/2017,10000<100000,45.0,"FRANCE, METROPOLITAN",30,CELLPHONE_ACCESSORY,ELECTRONICS,<10,-
2,NORMAL,NaN,False,NaN,0,0,0,1,PLATINUM,2013,...,58,5/2017,10000<100000,43.0,CHINA,-1,TOYS,BABY,<10,NOT_RECEIVED
3,RECOMMANDE,NaN,True,5<20,1,0,0,0,UNSUBSCRIBED,2007,...,31,5/2017,10000<100000,44.0,"FRANCE, METROPOLITAN",2,GARDEN_TOOLS,WHITE,50<100,-
4,RECOMMANDE,NaN,False,NaN,1,0,1,0,PLATINUM,2010,...,93,9/2017,1000<10000,44.0,CHINA,-1,MODEL,BABY,1000<5000,WITHDRAWAL


In [35]:
from sklearn.pipeline import  FeatureUnion

In [57]:
combined_features = FeatureUnion([("splitter", split)])

In [58]:
# Use combined features to transform dataset:
pd.DataFrame(combined_features.transform(TX_data)).head()

,0,1,2,3,4,5,6,7,8,9,...,17,18,19,20,21,22,23,24,25,26
0,NORMAL,False,1,0,1,0,UNSUBSCRIBED,2015,1992,34,...,NaN,NaN,10000,100000,,10,,5,NaN,NaN
1,NORMAL,False,1,0,0,0,UNSUBSCRIBED,2013,1952,77,...,NaN,NaN,10000,100000,,10,,5,NaN,NaN
2,NORMAL,False,0,0,0,1,PLATINUM,2013,1991,58,...,NaN,NaN,10000,100000,,10,50,100,NaN,NaN
3,RECOMMANDE,True,1,0,0,0,UNSUBSCRIBED,2007,1955,31,...,5,20,10000,100000,50,100,500,None,NaN,NaN
4,RECOMMANDE,False,1,0,1,0,PLATINUM,2010,1984,93,...,NaN,NaN,1000,10000,1000,5000,5,20,NaN,NaN


## Is Claim

Attention, c'est un peu dangereux d'utiliser la target variable pour faire du feature engineering.

Mais ici, je veux créer des "cartes d'identité" par vendeur et acheteur, pour reperer les bad guys. Donc je dois connaitre le taux de rejet de chacun. Ce que je vais faire plus tard.

In [ ]:
# Fonction qui prend une ligne de la BDD en entrée, et dit "Si c'est - alors on renvoit 0, sinon 1
def b(row):
        return 0 if row['CLAIM_TYPE'] == '-' else 1;

In [ ]:
# Je l'applique à la base de données
TX_data['IS_CLAIM'] = TX_data.apply(b, axis=1)

## Shipping Method

Regroupement des méthodes de shipping rares sous la même catégorie.

Objectif = Diminuer la complexité de la variable

In [ ]:
#TX_data["SHIPPING_MODE"].value_counts().head(10).plot.bar()

In [ ]:
list_rare_delivery_type = ['EXPRESS_DELIVERY',
                          'SO_RECOMMANDE',
                          'MONDIAL_RELAY',
                          'MONDIAL_RELAY_PREPAYE',
                          'SO_POINT_RELAIS',
                          'CHRONOPOST',
                          'PICKUP',
                          'Kiala']
TX_data["SHIPPING_MODE"].replace(to_replace=list_rare_delivery_type,
                                 value="RARE_TYPE",
                                 inplace=True)
del list_rare_delivery_type;

In [ ]:
TX_data = pd.merge(TX_data, 
                   pd.get_dummies(TX_data["SHIPPING_MODE"]), 
                   left_index=True, 
                   right_index=True)

In [ ]:
TX_data.drop("SHIPPING_MODE",
             axis=1,
             inplace=True)

## Customer Premium

Regroupement des types de comptes payant sous la même catégorie.

Objectif = Diminuer la complexité de la variable

In [ ]:
#TX_data["PRICECLUB_STATUS"].value_counts().head(10).plot.bar()

In [ ]:
list_rare_delivery_type= ['PLATINUM', 
                          'SILVER', 
                          'GOLD']
TX_data["PRICECLUB_STATUS"].replace(to_replace=list_rare_delivery_type, 
                                 value="PREMIUM", 
                                 inplace=True)
del list_rare_delivery_type

In [ ]:
TX_data = pd.merge(TX_data, pd.get_dummies(TX_data["PRICECLUB_STATUS"]), left_index=True, right_index=True)
TX_data.drop("PRICECLUB_STATUS", axis=1, inplace=True)

## Trajets longs

Flag pour signaler un envoi international

Objectif = souligner les courts trajets

In [ ]:
#TX_data["SELLER_COUNTRY"].value_counts().head(10).plot.bar()

In [ ]:
def f(row):
    long_haul_list=["CHINA","HONG KONG", "UNITED STATES"]
    return 1 if row['SELLER_COUNTRY'] in long_haul_list else 0;

In [ ]:
TX_data['LONG_HAUL'] = TX_data.apply(f, axis=1)

## Stats par Pays

To do : Détecter les pays à problèmes

In [ ]:
description_product_country = TX_data['IS_CLAIM'].groupby(TX_data['SELLER_COUNTRY']).describe().reset_index()

In [ ]:
colnames_country=["SELLER_COUNTRY","Count_country","Mean_Claims_country","STD_Claims_country","t_1","t_2","t_3","t_4","t_5"]

description_product_country.columns = colnames_country
description_product_country.drop(["t_1","t_2","t_3","t_4","t_5"], axis=1, inplace=True)
description_product_country["Count_Claims_country"] = description_product_country["Mean_Claims_country"] * description_product_country["Count_country"]

In [ ]:
TX_data=pd.merge(TX_data, description_product_country, on='SELLER_COUNTRY', how='outer')

del description_product_country, colnames_country

In [ ]:
#TX_data.drop(columns="SELLER_COUNTRY" ,axis=1,inplace=True)

## Type de produit

To do : Détecter les produits fragiles

Objectif = souligner cet effet

In [ ]:
#TX_data["PRODUCT_TYPE"].value_counts().head(10).plot.bar()

In [ ]:
description_product_type=TX_data['IS_CLAIM'].groupby(TX_data['PRODUCT_TYPE']).describe().reset_index()

In [ ]:
colnames_product_type=["PRODUCT_TYPE","Count_product_type","Mean_Claims_product_type","STD_Claims_product_type","t_1","t_2","t_3","t_4","t_5"]

description_product_type.columns=colnames_product_type
description_product_type.drop(["t_1","t_2","t_3","t_4","t_5"],axis=1,inplace=True)
description_product_type["Count_Claims_product_type"]=description_product_type["Mean_Claims_product_type"]*description_product_type["Count_product_type"]

In [ ]:
TX_data=pd.merge(TX_data, description_product_type, on='PRODUCT_TYPE', how='outer')
del description_product_type, colnames_product_type

In [ ]:
TX_data.drop(columns="PRODUCT_TYPE" ,axis=1,inplace=True)

## Création d'un ID par client 

Objectif = créer une base de données client

In [ ]:
import hashlib

In [ ]:
def idBuyerCreator(row):
        return hashlib.md5(
            (str(row['REGISTRATION_DATE'])+
             str(row["BUYER_DEPARTMENT"])+
             str(row["BUYER_BIRTHDAY_DATE"])
            ).encode()
        ).hexdigest()

In [ ]:
TX_data['BUYER_ID'] = TX_data.apply(idBuyerCreator, axis=1)

## Calcul du taux de claims par client

Objectif = Créer de nouvelles features

In [ ]:
description_buyer=TX_data['IS_CLAIM'].groupby(TX_data['BUYER_ID']).describe().reset_index()

In [ ]:
colnames_buyer=["BUYER_ID","Count_Buys","Mean_Claims_Buys","STD_Claims_Buys","t_1","t_2","t_3","t_4","t_5"]

description_buyer.columns=colnames_buyer
description_buyer.drop(["t_1","t_2","t_3","t_4","t_5"],axis=1,inplace=True)
description_buyer["Count_Claims_Buy"]=description_buyer["Mean_Claims_Buys"]*description_buyer["Count_Buys"]

In [ ]:
TX_data=pd.merge(TX_data, description_buyer, on='BUYER_ID', how='outer')
del description_buyer, colnames_buyer

## Création d'un ID par vendeur

Objectif = créer une base de données vendeur

In [ ]:
def idSellCreator(row):
        return hashlib.md5(
            (str(row["SELLER_COUNTRY"])+
             str(row["SELLER_DEPARTMENT"])+
             str(row["SELLER_SCORE_AVERAGE"])
            ).encode()
        ).hexdigest()

In [ ]:
TX_data['SELLER_ID'] = TX_data.apply(idSellCreator, axis=1)

## Calcul du taux de claims par vendeur

Objectif = Créer de nouvelles features

In [ ]:
description_sellers=TX_data['IS_CLAIM'].groupby(TX_data['SELLER_ID']).describe().reset_index()

In [ ]:
colnames_sell=["SELLER_ID","Count_Sells","Mean_Claims_Sells","STD_Claims_Sells","t_1","t_2","t_3","t_4","t_5"]

description_sellers.columns=colnames_sell
description_sellers.drop(["t_1","t_2","t_3","t_4","t_5"],axis=1,inplace=True)
description_sellers["Count_Claims_Sells"]=description_sellers["Mean_Claims_Sells"]*description_sellers["Count_Sells"]

In [ ]:
TX_data=pd.merge(TX_data, description_sellers, on='SELLER_ID', how='outer')
del description_sellers, colnames_sell

## Fusion des bases de données

Objectif = Yolo

In [ ]:
list_to_drop=["IS_CLAIM","BUYER_ID","SELLER_ID"]

In [ ]:
TX_data.drop(list_to_drop,inplace=True,axis=1)
del list_to_drop

In [ ]:
#TX_data.info()

## On coupe

Objectif = Créer un jeu de données train et test

In [ ]:
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(TX_data, 
                                       test_size=0.3, 
                                       random_state=RANDOM_SEED, 
                                       stratify=TX_data["CLAIM_TYPE"]
                                      )
del TX_data;

## Preprocess

Objectif = Rendre joli tout ça

In [ ]:
def datapreprocess(data):
    data=data.apply(pd.to_numeric, errors='ignore')
    
    # Y and X
    Y=data["CLAIM_TYPE"]
    X=data.drop("CLAIM_TYPE", axis=1,inplace=False)
    # Exclude Objets
    X=X.select_dtypes(exclude=['object'])
    
    # Work on fare
    from sklearn.preprocessing import Imputer
    imp = Imputer(missing_values='NaN',strategy='median', axis=1)
    X=pd.DataFrame(imp.fit_transform(X),columns=X.columns.values)
 
    return X, Y

In [ ]:
X_train,Y_train=datapreprocess(train_set)
X_test, Y_test=datapreprocess(test_set)

del train_set, test_set;
gc.collect()

# MODEL!

## Scaler

Inutile pour les arbres, mais utiles pour les autres

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
scaler = StandardScaler()

# PCA

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
pca = PCA()

# Feature Union

In [ ]:
from sklearn.pipeline import FeatureUnion

from sklearn.decomposition import KernelPCA
estimators = [('linear_pca', PCA(n_components=5)), 
                  ('kernel_pca', KernelPCA(n_components=5))]
combined = FeatureUnion(estimators)

In [ ]:
#combined.fit_transform(X_train)

# Feature Selection

## Selection des variables avec une variance faible

Ici, on vire tout ce qu'il y a une probabilité de 0.8 d'être la 

In [ ]:
from sklearn.feature_selection import VarianceThreshold

In [ ]:
sel_vt = VarianceThreshold(threshold=(.8 * (1 - .8)))

## Selection des K meilleures variables d'après le test de Chi2

In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

In [ ]:
Nb_Var = 15

In [ ]:
Chi_select = SelectKBest(chi2, k=Nb_Var)
del Nb_Var

## Selection par rapport à un modèle

In [ ]:
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier

### Modèle de Selection

In [ ]:
clf = RandomForestClassifier(n_jobs=-1, 
                             random_state=RANDOM_SEED)

### Contruction

In [ ]:
sfm_clf = SelectFromModel(clf)
del clf

# Metriques

D'abord, notre métrique à nous

### Calcul l'AUC de chaque classe

In [ ]:
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelBinarizer

In [ ]:
def multiclass_roc_auc_score(truth, pred):
    lb = LabelBinarizer()
    lb.fit(truth)
    return roc_auc_score(lb.transform(truth), lb.transform(pred), average="weighted")

### precision_class_AUC (pas safe), est utilisée pour le scoring de modèle (version adaptée de précédement)

In [ ]:
def precision_class_AUC(estimator, X, y):
    return multiclass_roc_auc_score(y, estimator.predict(X))

## Model!

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rnd_clf = RandomForestClassifier(n_jobs=-1, 
                                 random_state=RANDOM_SEED)

# Et on pipeline

In [ ]:
from sklearn.pipeline import Pipeline

In [ ]:
clf = Pipeline([
    #('scaler', scaler),  
    #('threshold', sel_vt), 
    #('SelectKBest', Chi_select), 
    #('reduce_dim', pca),
    #('feature_union', combined),
    ('feature_selection', sfm_clf),
    ('classification', rnd_clf)
])


clf.fit(X_train, Y_train);
y_pred_rf = clf.predict(X_test)
multiclass_roc_auc_score(Y_test, y_pred_rf)

In [ ]:
pd.DataFrame(y_pred_rf).head()

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
def plot_confusion_matrix(matrix):
    """If you prefer color and a colorbar"""
    fig = plt.figure(figsize=(8,8))
    ax = fig.add_subplot(111)
    cax = ax.matshow(matrix)
    fig.colorbar(cax)
    
conf_mx = confusion_matrix(Y_test, y_pred_rf)
plot_confusion_matrix(conf_mx)

## ATTENTION: SUPER LOOOOOOOOOOONG

In [ ]:
from sklearn.model_selection import cross_val_score

In [ ]:
scores = cross_val_score(n_jobs=-1,
                         X=X_train,
                         y=Y_train,
                         cv=3,
                         scoring=precision_class_AUC,
                         estimator=rnd_clf,
                         verbose=1)

print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
params = {'max_leaf_nodes': [4, 8],
          'max_depth' : [2, 4], 
          'n_estimators': list(range(10, 50, 10)),
          'min_samples_leaf': list(range(1, 3)),
          'min_samples_split' : list(range(2, 3)),
         }

In [ ]:
rf_gs_cv = GridSearchCV(RandomForestClassifier(random_state=RANDOM_SEED, oob_score=True),
                              params,
                              scoring=precision_class_AUC,
                              n_jobs=-1, 
                              verbose=1)

In [ ]:
rf_gs_cv.fit(X_train, Y_train)
rf_gs_cv.best_estimator_

In [ ]:
y_pred_cv_rf = rf_gs_cv.predict(X_test) 

In [ ]:
multiclass_roc_auc_score(Y_test, y_pred_cv_rf)

In [ ]:
conf_mx = confusion_matrix(Y_test, y_pred_cv_rf)
plot_confusion_matrix(conf_mx)

# Random Research of Parameters

In [ ]:
print(__doc__)
from time import time
from scipy.stats import randint as sp_randint

from sklearn.model_selection import RandomizedSearchCV

In [ ]:
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

In [ ]:
# build a classifier
clf = RandomForestClassifier(n_jobs=-1, 
                             random_state=RANDOM_SEED, 
                             verbose=0, 
                             warm_start=False,
                            oob_score=False)

In [ ]:
params_cv_long = {'bootstrap':[True, False], 
                  'max_depth':[2, 3, None], 
                  'max_features':sp_randint(1, 11), 
                  'max_leaf_nodes':sp_randint(2, 11),
                  'min_samples_leaf':sp_randint(1, 11), 
                  'min_samples_split':sp_randint(2, 11),
                  'n_estimators':sp_randint(5, 30), 
                 }

In [ ]:
# run randomized search
n_iter_search = 50
random_search = RandomizedSearchCV(clf, 
                                   param_distributions=params_cv_long,
                                   n_iter=n_iter_search,
                                   scoring=precision_class_AUC)

In [ ]:
start = time()
random_search.fit(X=X_train,y=Y_train)

In [ ]:
print("RandomizedSearchCV took %.2f seconds for %d candidates"
      " parameter settings." % ((time() - start), n_iter_search))
report(random_search.cv_results_)